In [35]:
# import dill
# dill.dump_session('notebook_env.db')
# import dill
# dill.load_session('notebook_env.db')

In [101]:
import pke
from tqdm.notebook import tqdm
import ipywidgets as widgets
import os
import pandas as pd
import json
import spacy
import string
from nltk.corpus import stopwords
import re
import warnings
warnings.filterwarnings("ignore")
import string

In [143]:
with open('mj.json') as json_file:
    mj = json.load(json_file)
    
with open('ah.json') as json_file:
    ah = json.load(json_file)

with open('se.json') as json_file:
    se = json.load(json_file)

In [144]:
def preprocessor(text):
    if type(text)!=str:
        text = str(text)
    nlp = spacy.load("en_core_web_lg")
    doc = nlp(text)
    lemma_list = []
    for token in doc:
        if token.is_alpha is True:
            lemma_list.append(token.lemma_)
    return_list=[]
    for i in range(len(lemma_list)):
        temp_text = re.sub('<[^>]*>', '', lemma_list[i])
        temp_text = re.sub('[\W]+', '', temp_text.lower())
        return_list.append(temp_text)
    return(return_list)

In [145]:
def evaluate_keywords(proposed,groundtruth):
    
    proposed_set = set(proposed)
    true_set = set(groundtruth)
    
    true_positives = len(proposed_set.intersection(true_set))
    
    if len(proposed_set)==0:
        precision = 0
    
    else:
        precision = true_positives/float(len(proposed)) 
    
    if len(true_set)==0:
        recall = 0
    
    else:
        recall = true_positives/float(len(true_set))
        
    if precision + recall > 0:
        f1 = 2*precision*recall/float(precision + recall)
    
    else:
        f1 = 0
    
    return (precision, recall, f1)

In [147]:
def evaluate(dataset, model):
#dataset = mj,ah,se
#model = multipartiterank, topicrank, textrank, positionrank
    final_result_5 = []
    final_result_10 = []
    result = []
    p_5 = 0
    p_10 = 0
    r_5 = 0
    r_10 = 0
    f_10 = 0
    f_5=0
    for i in tqdm(dataset):
        a = i['keywords'][0]
        b = i[str(model)]
        x = evaluate_keywords(preprocessor(b[0]),preprocessor(a))
        y = evaluate_keywords(preprocessor(b[1]),preprocessor(a))
        p_5 = p+ x[0]
        r_5 = r+ x[1]
        f_5 = f+ x[2]
        p_10 = p+ y[0]
        r_10 = r+ y[1]
        f_10 = f+ y[2]
    final_result_5.append(p_5)
    final_result_5.append(r_5)
    final_result_5.append(f_5)
    final_result_10.append(p_10)
    final_result_10.append(r_10)
    final_result_10.append(f_10)
    result.append(final_result_5)
    result.append(final_result_10)
    return result